In [1]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Imports
import os
import cv2
import sys
import time
import json
import copy
import cortex
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.spatial import Delaunay

# Personal imports
sys.path.append("{}/../../../analysis_code/utils".format(os.getcwd()))
from plot_utils import *
from pycortex_utils import draw_cortex, set_pycortex_config_file, load_surface_pycortex, create_colormap, get_rois
from surface_utils import load_surface

In [2]:
# Directories
main_dir = '/Users/uriel/disks/meso_shared'
project_dir = 'RetinoMaps'

In [3]:
with open('../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
subjects = analysis_info['subjects']

In [4]:
# Set pycortex db and colormaps
cortex_dir = "{}/{}/derivatives/pp_data/cortex".format(main_dir, project_dir)
set_pycortex_config_file(cortex_dir)

# Webgl port
port_num = 25000

# pRF

## Zoom on ROI

In [5]:
prf_task_name = analysis_info['prf_task_name']
maps_names_css = analysis_info['maps_names_css']
maps_names_pcm = analysis_info['maps_names_pcm']
maps_names_css_stats = analysis_info['maps_names_css_stats']
alpha_range = analysis_info["alpha_range"]

In [7]:
# Settings
pycortex_subject = 'sub-170k'
format_ = '170k'
extension = 'dtseries.nii'
suffix = 'Sac_Pur'
flatmaps_dir = '/Users/uriel/Library/CloudStorage/Dropbox/retinomaps/figures/material/flatmaps/all_subjects_prf'


save_svg = False

vmin = 0 
vmax = 0.4
cmap_steps = 255

cmap_polar, cmap_uni, cmap_ecc_size = 'hsv', 'Reds', 'Spectral'
col_offset = 1.0/14.0
cmap_steps = 255

# plot scales
pcm_scale = [0, 10]
size_scale = [0, 10]
ecc_scale = [0, 10]
rsq_scale = [0, 1]
polar_scale = [0, 1]
n_scale = [0, 2]

for subject in subjects : 
    print(subject)
    #### Load Data ####
    all_idx = 0
    # Maps settings 
    for idx, col_name in enumerate(maps_names_css + maps_names_css_stats + maps_names_pcm):
        exec("{}_idx = idx".format(col_name))

    # Define directories and fn
    prf_dir = "{}/{}/derivatives/pp_data/{}/{}/prf".format(main_dir, project_dir, subject, format_)
    prf_deriv_dir = "{}/prf_derivatives".format(prf_dir)
    intertask_dir = "{}/{}/derivatives/pp_data/{}/{}/intertask".format(
    main_dir, project_dir, subject,format_)

    # Derivatives
    deriv_median_fn = '{}/{}_task-{}_fmriprep_dct_avg_prf-deriv_css_loo-median.dtseries.nii'.format(
        prf_deriv_dir, subject, prf_task_name)
    deriv_results = load_surface_pycortex(brain_fn=deriv_median_fn)
    deriv_mat = deriv_results['data_concat']

    # Stats
    stats_median_fn = '{}/{}_task-{}_fmriprep_dct_avg_prf-stats_loo-median.dtseries.nii'.format(
        prf_deriv_dir, subject, prf_task_name)
    stats_results = load_surface_pycortex(brain_fn=stats_median_fn)
    stats_mat = stats_results['data_concat']

    # pRF CM
    pcm_median_fn = '{}/{}_task-{}_fmriprep_dct_avg_prf-pcm_css_loo-median.dtseries.nii'.format(
        prf_deriv_dir, subject, prf_task_name)
    pcm_results = load_surface_pycortex(brain_fn=pcm_median_fn)
    pcm_mat = pcm_results['data_concat']

    # intertask results 
    stats_avg_fn = '{}/{}_intertask_{}.dtseries.nii'.format(
    intertask_dir, subject, suffix)
    results_stats = load_surface_pycortex(brain_fn=stats_avg_fn)
    intertask_mat = results_stats['data_concat']
    mask_intertask = intertask_mat[all_idx, :] == 7

    # Combine mat
    all_deriv_mat = np.concatenate((deriv_mat, stats_mat, pcm_mat))

    # Threshold mat
    all_deriv_mat_th = all_deriv_mat
    amp_down = all_deriv_mat_th[amplitude_idx,...] > 0
    rsq_down = all_deriv_mat_th[prf_loo_r2_idx,...] >= analysis_info['rsqr_th']
    size_th_down = all_deriv_mat_th[prf_size_idx,...] >= analysis_info['size_th'][0]
    size_th_up = all_deriv_mat_th[prf_size_idx,...] <= analysis_info['size_th'][1]
    ecc_th_down = all_deriv_mat_th[prf_ecc_idx,...] >= analysis_info['ecc_th'][0]
    ecc_th_up = all_deriv_mat_th[prf_ecc_idx,...] <= analysis_info['ecc_th'][1]
    n_th_down = all_deriv_mat_th[prf_n_idx,...] >= analysis_info['n_th'][0]
    n_th_up = all_deriv_mat_th[prf_n_idx,...] <= analysis_info['n_th'][1]
    if analysis_info['stats_th'] == 0.05: stats_th_down = all_deriv_mat_th[corr_pvalue_5pt_idx,...] <= 0.05
    elif analysis_info['stats_th'] == 0.01: stats_th_down = all_deriv_mat_th[corr_pvalue_1pt_idx,...] <= 0.01
    all_th = np.array((amp_down,
                       rsq_down,
                       size_th_down,size_th_up, 
                       ecc_th_down, ecc_th_up,
                       n_th_down, n_th_up,
                       stats_th_down
                      )) 
    all_deriv_mat[prf_loo_r2_idx, np.logical_and.reduce(all_th)==False]=0 # put this to zero to not plot it

    # === Retrieve ROIs and define masks ===
    rois_dic = get_rois(
        subject=subject, return_concat_hemis=True, return_hemi=None,
        rois=None, mask=True, atlas_name='mmp_group', surf_size='59k'
    )

    frontal_mask = rois_dic['iPCS'] | rois_dic['sPCS'] | rois_dic['mPCS']
    parietal_mask = rois_dic['iIPS'] | rois_dic['sIPS']
    frontal_parietal_mask = frontal_mask | parietal_mask

    # === Generate colormap ===
    base_cmap = plt.cm.get_cmap('viridis')
    val = np.linspace(0, 1, cmap_steps, endpoint=False)
    colmap = colors.LinearSegmentedColormap.from_list('custom_colormap', base_cmap(val), N=cmap_steps)

     # === Préparer dictionnaire de données par tâche ===
    pol_comp_num = all_deriv_mat[polar_real_idx,...] + 1j * all_deriv_mat[polar_imag_idx,...]
    polar_ang = np.angle(pol_comp_num)
    ang_norm = (polar_ang + np.pi) / (np.pi * 2.0)
    ang_norm = np.fmod(ang_norm + col_offset,1)    

    prf_param_data = {
        'looR2': all_deriv_mat[prf_loo_r2_idx, :],
        'polar': ang_norm,
        'pcm': all_deriv_mat[pcm_median_idx, :],
        'ecc': all_deriv_mat[prf_ecc_idx, :],
        'size': all_deriv_mat[prf_size_idx, :]
    }

    zooms = {
        'IPEF_L': {'hemi': 'left',  'roi': ['iIPS'], 'xmin_add': -60, 'xmax_add': 10,  'ymin_add': -50, 'ymax_add': 50},
        'PCEF_L': {'hemi': 'left',  'roi': ['sPCS'], 'xmin_add': -60, 'xmax_add': 10,  'ymin_add': -70, 'ymax_add': 30},
        'IPEF_R': {'hemi': 'right', 'roi': ['iIPS'], 'xmin_add': -20, 'xmax_add': 50,  'ymin_add': -50, 'ymax_add': 50},
        'PCEF_R': {'hemi': 'right', 'roi': ['sPCS'], 'xmin_add': -10, 'xmax_add': 60,  'ymin_add': -70, 'ymax_add': 30},
    }
    target_width = 125
    target_height = 200
    # === Appliquer alpha et créer figures pour chaque tâche ===
    for task_name, pRF_data in prf_param_data.items():
        if task_name == 'looR2': vmin, vmax, cmap = rsq_scale[0], rsq_scale[1], cmap_uni
        elif task_name == 'polar': vmin, vmax, cmap = polar_scale[0], polar_scale[1], cmap_polar
        elif task_name == 'pcm': vmin, vmax, cmap = pcm_scale[0], pcm_scale[1], cmap_ecc_size
        elif task_name == 'ecc': vmin, vmax, cmap = ecc_scale[0], ecc_scale[1], cmap_ecc_size
        elif task_name == 'size': vmin, vmax, cmap = size_scale[0], size_scale[1], cmap_ecc_size

        try: base = plt.cm.get_cmap(cmap)
        except: base = cortex.utils.get_cmap(cmap)

        if '_alpha' in cmap: base.colors = base.colors[1,:,:]
        val = np.linspace(0, 1, cmap_steps, endpoint=False)

        colmap = colors.LinearSegmentedColormap.from_list('my_colmap', base(val), N=cmap_steps)

        # Calcul alpha (transparence) selon le range
        alpha_data = (all_deriv_mat[prf_loo_r2_idx, :] - alpha_range[0]) / (alpha_range[1] - alpha_range[0])
        alpha_data = np.clip(alpha_data, 0, 1)
        alpha_data[~frontal_parietal_mask] = 0

        # === Convertir en RGB et appliquer alpha ===
        norm_data = ((pRF_data - vmin) / (vmax - vmin)) * cmap_steps
        rgb_data = colmap(norm_data.astype(int)) * 255.0
        alpha_scaled = alpha_data * 255.0

        # === Créer l'objet cortex.VertexRGB ===
        braindata = cortex.VertexRGB(
            red=rgb_data[..., 0].astype(np.uint8),
            green=rgb_data[..., 1].astype(np.uint8),
            blue=rgb_data[..., 2].astype(np.uint8),
            subject=pycortex_subject,
            alpha=alpha_scaled.astype(np.uint8)
        ).blend_curvature(alpha_scaled)

        # === Génération des figures pour chaque zoom ===
        for zoom, cfg in zooms.items():
            hemi = cfg['hemi']
            roi = cfg['roi']
            xmin_add, xmax_add = cfg['xmin_add'], cfg['xmax_add']
            ymin_add, ymax_add = cfg['ymin_add'], cfg['ymax_add']

            # Get ROI vertex indices
            roi_verts = get_rois(
                subject=subject, return_concat_hemis=True, return_hemi=False,
                rois=roi, mask=False, atlas_name='mmp_group', surf_size='59k'
            )
            roi_map = cortex.Vertex.empty(pycortex_subject)
            roi_map.data[roi_verts[roi[0]]] = 1

            # Get flat surface coordinates
            (lflatpts, _), (rflatpts, _) = cortex.db.get_surf(pycortex_subject, "flat", nudge=True)
            sel_pts = {'left': lflatpts, 'right': rflatpts}[hemi]
            roi_pts = sel_pts[np.nonzero(getattr(roi_map, hemi))[0], :2]

            # Bounds and adjustment
            xmin = roi_pts[:, 0].min() + xmin_add
            xmax = roi_pts[:, 0].max() + xmax_add
            ymin = roi_pts[:, 1].min() + ymin_add
            ymax = roi_pts[:, 1].max() + ymax_add

            if (xmax - xmin) < target_width:
                xmax += target_width - (xmax - xmin)
            else:
                xmax = xmin + target_width

            if (ymax - ymin) < target_height:
                ymax += target_height - (ymax - ymin)
            else:
                ymax = ymin + target_height

            # Génération et sauvegarde
            fig = cortex.quickshow(
                braindata, depth=1, thick=1, sampler='nearest',
                with_curvature=True, nanmean=True, with_labels=True,
                with_colorbar=False, with_borders=True,
                curvature_brightness=1, curvature_contrast=0.25,
                with_figure=True
            )
            ax = fig.axes[0]
            ax.axis([xmin, xmax, ymin, ymax])
            ax.set_position([0, 0, 1, 1])
            ax.axis('off')
            fig.set_size_inches(5, 5)

            out_path = "{}/{}_{}_{}.pdf".format(flatmaps_dir, subject, task_name, zoom)
            plt.savefig(out_path, bbox_inches='tight', pad_inches=0, dpi=500)
            plt.close(fig)



sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-11
sub-12
sub-13
sub-14
sub-17
sub-20
sub-21
sub-22
sub-23
sub-24
sub-25


# ROIs

In [5]:
subject = 'sub-11'
format_ = '170k'
pycortex_subject = 'sub-170k'
save_svg = False

rois_dir = "{}/{}/derivatives/pp_data/{}/{}/rois".format(main_dir, project_dir, subject,format_)
roi_fn = '{}/{}_rois.dtseries.nii'.format(rois_dir, subject)
results = load_surface_pycortex(brain_fn=roi_fn)
roi_mat = results['data_concat']

In [6]:
cmap = 'rois_colors'
colormap_dict = {'n/a': (255, 255, 255),
                 'V1': (243, 231, 155),
                 'V2': (250, 196, 132),
                 'V3': (248, 160, 126),
                 'V3AB': (235, 127, 134),
                 'LO': (150, 0, 90), 
                 'VO': (0, 0, 200),
                 'hMT+': (0, 25, 255),
                 'iIPS': (0, 152, 255),
                 'sIPS': (44, 255, 150),
                 'iPCS': (151, 255, 0),
                 'sPCS': (255, 234, 0),
                 'mPCS': (255, 111, 0)}

In [7]:
flatmaps_dir = '/Users/uriel/Library/CloudStorage/Dropbox/retinomaps/figures/material/flatmaps'
colormap_name = 'rois_colors'
rois_opacity = 0.8
vmin = 0
vmax = 12
cmap_steps = len(colormap_dict)

val = np.linspace(0, 1, cmap_steps, endpoint=False)
base = cortex.utils.get_cmap(cmap)    
colmap = colors.LinearSegmentedColormap.from_list('my_colmap', base(val), N=cmap_steps)



rois_dic = get_rois(subject=subject, return_concat_hemis=True, return_hemi=None, rois=None, mask=True, atlas_name='mmp_group', surf_size='59k')
frontal_mask = rois_dic['iPCS'] | rois_dic['sPCS'] | rois_dic['mPCS']
parietal_mask = rois_dic['iIPS'] | rois_dic['sIPS']
parietal_frontal_mask = rois_dic['iPCS'] | rois_dic['sPCS'] | rois_dic['mPCS'] | rois_dic['iIPS'] | rois_dic['sIPS']

rois_mask = parietal_frontal_mask

alpha_mat = roi_mat*0+rois_opacity
alpha_mat[roi_mat==0]=0

rois_mask = rois_mask.reshape(1,-1)
alpha_mat[~rois_mask] = 0

zooms = {
    'IPEF_L': {'hemi': 'left',  'roi': ['iIPS'], 'xmin_add': -60, 'xmax_add': 10,  'ymin_add': -50, 'ymax_add': 50},
    'PCEF_L': {'hemi': 'left',  'roi': ['sPCS'], 'xmin_add': -60, 'xmax_add': 10,  'ymin_add': -70, 'ymax_add': 30},
    'IPEF_R': {'hemi': 'right', 'roi': ['iIPS'], 'xmin_add': -20, 'xmax_add': 50,  'ymin_add': -50, 'ymax_add': 50},
    'PCEF_R': {'hemi': 'right', 'roi': ['sPCS'], 'xmin_add': -10, 'xmax_add': 60,  'ymin_add': -70, 'ymax_add': 30},
}
target_width = 125
target_height = 200

# === Convertir en RGB et appliquer alpha ===
norm_data = ((roi_mat - vmin) / (vmax - vmin)) * cmap_steps
rgb_data = colmap(norm_data.astype(int)) * 255.0
alpha_scaled = alpha_mat * 255.0

# === Créer l'objet cortex.VertexRGB ===
braindata = cortex.VertexRGB(
    red=rgb_data[..., 0].astype(np.uint8),
    green=rgb_data[..., 1].astype(np.uint8),
    blue=rgb_data[..., 2].astype(np.uint8),
    subject=pycortex_subject,
    alpha=alpha_scaled.astype(np.uint8)
).blend_curvature(alpha_scaled)

# === Génération des figures pour chaque zoom ===
for zoom, cfg in zooms.items():
    hemi = cfg['hemi']
    roi = cfg['roi']
    xmin_add, xmax_add = cfg['xmin_add'], cfg['xmax_add']
    ymin_add, ymax_add = cfg['ymin_add'], cfg['ymax_add']

    # Get ROI vertex indices
    roi_verts = get_rois(
        subject=subject, return_concat_hemis=True, return_hemi=False,
        rois=roi, mask=False, atlas_name='mmp_group', surf_size='59k'
    )
    roi_map = cortex.Vertex.empty(pycortex_subject)
    roi_map.data[roi_verts[roi[0]]] = 1

    # Get flat surface coordinates
    (lflatpts, _), (rflatpts, _) = cortex.db.get_surf(pycortex_subject, "flat", nudge=True)
    sel_pts = {'left': lflatpts, 'right': rflatpts}[hemi]
    roi_pts = sel_pts[np.nonzero(getattr(roi_map, hemi))[0], :2]

    # Bounds and adjustment
    xmin = roi_pts[:, 0].min() + xmin_add
    xmax = roi_pts[:, 0].max() + xmax_add
    ymin = roi_pts[:, 1].min() + ymin_add
    ymax = roi_pts[:, 1].max() + ymax_add

    if (xmax - xmin) < target_width:
        xmax += target_width - (xmax - xmin)
    else:
        xmax = xmin + target_width

    if (ymax - ymin) < target_height:
        ymax += target_height - (ymax - ymin)
    else:
        ymax = ymin + target_height

    # Génération et sauvegarde
    fig = cortex.quickshow(
        braindata, depth=1, thick=1, sampler='nearest',
        with_curvature=True, nanmean=True, with_labels=True,
        with_colorbar=False, with_borders=True,
        curvature_brightness=1, curvature_contrast=0.25,
        with_figure=True
    )
    ax = fig.axes[0]
    ax.axis([xmin, xmax, ymin, ymax])
    ax.set_position([0, 0, 1, 1])
    ax.axis('off')
    fig.set_size_inches(5, 5)

    out_path = "{}/{}_roi_{}.pdf".format(flatmaps_dir, subject, zoom)
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0, dpi=500)
    plt.close(fig)


# GLM results

### Zoom on roi 

In [5]:
# Settings
pycortex_subject = 'sub-170k'
format_ = '170k'
extension = 'dtseries.nii'
prf_task_name = analysis_info['prf_task_name']
alpha_range = analysis_info["alpha_range"]
group_tasks = analysis_info['task_intertask']
flatmaps_dir = '/Users/uriel/Library/CloudStorage/Dropbox/retinomaps/figures/material/flatmaps/all_subjects'

suffix = 'Sac_Pur'
sac_task = 'SacLoc'
pur_task = 'PurLoc'
save_svg = False

vmin = 0 
vmax = 0.4
cmap_steps = 255

zooms = {
    'IPEF_L': {'hemi': 'left',  'roi': ['iIPS'], 'xmin_add': -60, 'xmax_add': 10,  'ymin_add': -50, 'ymax_add': 50},
    'PCEF_L': {'hemi': 'left',  'roi': ['sPCS'], 'xmin_add': -60, 'xmax_add': 10,  'ymin_add': -70, 'ymax_add': 30},
    'IPEF_R': {'hemi': 'right', 'roi': ['iIPS'], 'xmin_add': -20, 'xmax_add': 50,  'ymin_add': -50, 'ymax_add': 50},
    'PCEF_R': {'hemi': 'right', 'roi': ['sPCS'], 'xmin_add': -10, 'xmax_add': 60,  'ymin_add': -70, 'ymax_add': 30},
}

target_width = 125
target_height = 200

for subject in subjects : 
    print(subject)
    # Maps settings
    all_idx, pur_idx, sac_idx, pur_sac_idx, prf_idx, prf_pur_idx, prf_sac_idx, \
            prf_pur_sac_idx = 0,1,2,3,4,5,6,7

    slope_idx, intercept_idx, rvalue_idx, pvalue_idx, stderr_idx, \
        trs_idx, corr_pvalue_5pt_idx, corr_pvalue_1pt_idx = 0, 1, 2, 3, 4, 5, 6, 7

    # Set pycortex db and colormaps
    cortex_dir = "{}/{}/derivatives/pp_data/cortex".format(main_dir, project_dir)
    set_pycortex_config_file(cortex_dir)

    # Define/create colormap
    colormap_name = 'viridis'

    # Define directories and fn
    intertask_dir = "{}/{}/derivatives/pp_data/{}/{}/intertask".format(
        main_dir, project_dir, subject,format_)
    prf_stats_dir = "{}/{}/derivatives/pp_data/{}/{}/prf/prf_derivatives".format(
        main_dir, project_dir, subject,format_)
    glm_stats_dir = "{}/{}/derivatives/pp_data/{}/{}/glm/glm_derivatives".format(
        main_dir, project_dir,  subject,format_)

    stats_avg_fn = '{}/{}_intertask_{}.dtseries.nii'.format(
        intertask_dir, subject, suffix)
    prf_stats_avg_fn = '{}/{}_task-{}_fmriprep_dct_avg_prf-stats_loo-median.dtseries.nii'.format(
        prf_stats_dir, subject, prf_task_name)
    sac_stats_avg_fn = '{}/{}_task-{}_fmriprep_dct_avg_glm-stats_loo-median.dtseries.nii'.format(
        glm_stats_dir, subject, sac_task)
    pur_stats_avg_fn = '{}/{}_task-{}_fmriprep_dct_avg_glm-stats_loo-median.dtseries.nii'.format(
        glm_stats_dir, subject, pur_task)

    #  Load data
    results_stats = load_surface_pycortex(brain_fn=stats_avg_fn)
    intertask_mat = results_stats['data_concat']

    results_prf = load_surface_pycortex(brain_fn=prf_stats_avg_fn)
    prf_mat = results_prf['data_concat']

    results_sac = load_surface_pycortex(brain_fn=sac_stats_avg_fn)
    sac_mat = results_sac['data_concat']

    results_pur = load_surface_pycortex(brain_fn=pur_stats_avg_fn)
    pur_mat = results_pur['data_concat']

    # Compute R2 from R
    prf_mat[rvalue_idx,:] =  prf_mat[rvalue_idx,:]**2
    sac_mat[rvalue_idx,:] = sac_mat[rvalue_idx,:]**2
    pur_mat[rvalue_idx,:] = pur_mat[rvalue_idx,:]**2

    # threshold data
    # pRF
    prf_mat_corrected = copy.copy(prf_mat)
    prf_mat_corrected_th = prf_mat_corrected
    if analysis_info['stats_th'] == 0.05: stats_th_down = prf_mat_corrected_th[corr_pvalue_5pt_idx,...] <= 0.05
    elif analysis_info['stats_th'] == 0.01: stats_th_down = prf_mat_corrected_th[corr_pvalue_1pt_idx,...] <= 0.01
    prf_mat_corrected[rvalue_idx, stats_th_down==False]=0 # put this to zero to not plot it

    # SacLoc        
    sac_mat_corrected = copy.copy(sac_mat)
    sac_mat_corrected_th = sac_mat_corrected
    if analysis_info['stats_th'] == 0.05: stats_th_down = sac_mat_corrected_th[corr_pvalue_5pt_idx,...] <= 0.05
    elif analysis_info['stats_th'] == 0.01: stats_th_down = sac_mat_corrected_th[corr_pvalue_1pt_idx,...] <= 0.01
    sac_mat_corrected[rvalue_idx, stats_th_down==False]=0 # put this to zero to not plot it

    # PurLoc        
    pur_mat_corrected = copy.copy(pur_mat)
    pur_mat_corrected_th = pur_mat_corrected
    if analysis_info['stats_th'] == 0.05: stats_th_down = pur_mat_corrected_th[corr_pvalue_5pt_idx,...] <= 0.05
    elif analysis_info['stats_th'] == 0.01: stats_th_down = pur_mat_corrected_th[corr_pvalue_1pt_idx,...] <= 0.01
    pur_mat_corrected[rvalue_idx, stats_th_down==False]=0 # put this to zero to not plot it
    #  Creat R2 for the all flatmap
    rsq_all = np.zeros((intertask_mat[all_idx,...].shape))
    for vert, categorie in enumerate(intertask_mat[all_idx,...]):
        if categorie == 1: rsq_all[vert] = pur_mat_corrected[rvalue_idx,vert]

        elif categorie == 2: rsq_all[vert] = sac_mat_corrected[rvalue_idx,vert]

        elif categorie == 3: rsq_all[vert] = np.nanmedian([pur_mat_corrected[rvalue_idx,vert], 
                                                         sac_mat_corrected[rvalue_idx,vert]],
                                                        axis=0)

        elif categorie == 4: rsq_all[vert] = prf_mat_corrected[rvalue_idx,vert]

        elif categorie == 5: rsq_all[vert] = np.nanmedian([prf_mat_corrected[rvalue_idx,vert], 
                                                         pur_mat_corrected[rvalue_idx,vert]],
                                                        axis=0)

        elif categorie == 6: rsq_all[vert] = np.nanmedian([prf_mat_corrected[rvalue_idx,vert], 
                                                         sac_mat_corrected[rvalue_idx,vert]],
                                                        axis=0)

        elif categorie == 7: rsq_all[vert] = np.nanmedian([prf_mat_corrected[rvalue_idx,vert], 
                                                          sac_mat_corrected[rvalue_idx,vert], 
                                                          pur_mat_corrected[rvalue_idx,vert]], 
                                                        axis=0)

    #  Creat R2 for the pur_sac flatmap       
    rsq_pur_sac = np.zeros((intertask_mat[pur_sac_idx,...].shape))
    for vert, categorie in enumerate(intertask_mat[pur_sac_idx,...]):
        if categorie == 3: rsq_pur_sac[vert] = np.nanmedian([pur_mat_corrected[rvalue_idx,vert], 
                                                           sac_mat_corrected[rvalue_idx,vert]], 
                                                          axis=0)

    #  Creat R2 for the prf_pur flatmap               
    rsq_prf_pur = np.zeros((intertask_mat[prf_pur_idx,...].shape))
    for vert, categorie in enumerate(intertask_mat[prf_pur_idx,...]):
        if categorie == 5: rsq_prf_pur[vert] = np.nanmedian([pur_mat_corrected[rvalue_idx,vert], 
                                                           sac_mat_corrected[rvalue_idx,vert]], 
                                                          axis=0)

    #  Creat R2 for the prf_sac flatmap               
    rsq_prf_sac = np.zeros((intertask_mat[prf_sac_idx,...].shape))
    for vert, categorie in enumerate(intertask_mat[prf_sac_idx,...]):
        if categorie == 6: rsq_prf_sac[vert] = np.nanmedian([prf_mat_corrected[rvalue_idx,vert], 
                                                           sac_mat_corrected[rvalue_idx,vert]], 
                                                          axis=0)

    #  Creat R2 for the prf_pur_sac flatmap               
    rsq_prf_pur_sac = np.zeros((intertask_mat[all_idx,...].shape))
    for vert, categorie in enumerate(intertask_mat[all_idx,...]):
        if categorie == 7: rsq_prf_pur_sac[vert] = np.nanmean([prf_mat[rvalue_idx,vert], 
                                                               sac_mat_corrected[rvalue_idx,vert], 
                                                               pur_mat_corrected[rvalue_idx,vert]], 
                                                              axis=0)

    # === Retrieve ROIs and define masks ===
    rois_dic = get_rois(
        subject=subject, return_concat_hemis=True, return_hemi=None,
        rois=None, mask=True, atlas_name='mmp_group', surf_size='59k'
    )

    frontal_mask = rois_dic['iPCS'] | rois_dic['sPCS'] | rois_dic['mPCS']
    parietal_mask = rois_dic['iIPS'] | rois_dic['sIPS']
    frontal_parietal_mask = frontal_mask | parietal_mask

    # === Generate colormap ===
    base_cmap = plt.cm.get_cmap('viridis')
    val = np.linspace(0, 1, cmap_steps, endpoint=False)
    colmap = colors.LinearSegmentedColormap.from_list('custom_colormap', base_cmap(val), N=cmap_steps)
    
     # === Préparer dictionnaire de données par tâche ===
    task_data = {
        'pur': pur_mat_corrected[rvalue_idx, :],
        'sac': sac_mat_corrected[rvalue_idx, :],
        'prf': prf_mat_corrected[rvalue_idx, :],
        'prf_pur_sac': rsq_prf_pur_sac
    }

    # === Appliquer alpha et créer figures pour chaque tâche ===
    for task_name, rsq_data in task_data.items():

        # Calcul alpha (transparence) selon le range
        alpha_data = (rsq_data - alpha_range[0]) / (alpha_range[1] - alpha_range[0])
        alpha_data = np.clip(alpha_data, 0, 1)
        alpha_data[~frontal_parietal_mask] = 0

        # === Convertir en RGB et appliquer alpha ===
        norm_data = ((rsq_data - vmin) / (vmax - vmin)) * cmap_steps
        rgb_data = colmap(norm_data.astype(int)) * 255.0
        alpha_scaled = alpha_data * 255.0

        # === Créer l'objet cortex.VertexRGB ===
        braindata = cortex.VertexRGB(
            red=rgb_data[..., 0].astype(np.uint8),
            green=rgb_data[..., 1].astype(np.uint8),
            blue=rgb_data[..., 2].astype(np.uint8),
            subject=pycortex_subject,
            alpha=alpha_scaled.astype(np.uint8)
        ).blend_curvature(alpha_scaled)

        # === Génération des figures pour chaque zoom ===
        for zoom, cfg in zooms.items():
            hemi = cfg['hemi']
            roi = cfg['roi']
            xmin_add, xmax_add = cfg['xmin_add'], cfg['xmax_add']
            ymin_add, ymax_add = cfg['ymin_add'], cfg['ymax_add']

            # Get ROI vertex indices
            roi_verts = get_rois(
                subject=subject, return_concat_hemis=True, return_hemi=False,
                rois=roi, mask=False, atlas_name='mmp_group', surf_size='59k'
            )
            roi_map = cortex.Vertex.empty(pycortex_subject)
            roi_map.data[roi_verts[roi[0]]] = 1

            # Get flat surface coordinates
            (lflatpts, _), (rflatpts, _) = cortex.db.get_surf(pycortex_subject, "flat", nudge=True)
            sel_pts = {'left': lflatpts, 'right': rflatpts}[hemi]
            roi_pts = sel_pts[np.nonzero(getattr(roi_map, hemi))[0], :2]

            # Bounds and adjustment
            xmin = roi_pts[:, 0].min() + xmin_add
            xmax = roi_pts[:, 0].max() + xmax_add
            ymin = roi_pts[:, 1].min() + ymin_add
            ymax = roi_pts[:, 1].max() + ymax_add

            if (xmax - xmin) < target_width:
                xmax += target_width - (xmax - xmin)
            else:
                xmax = xmin + target_width

            if (ymax - ymin) < target_height:
                ymax += target_height - (ymax - ymin)
            else:
                ymax = ymin + target_height

            # Génération et sauvegarde
            fig = cortex.quickshow(
                braindata, depth=1, thick=1, sampler='nearest',
                with_curvature=True, nanmean=True, with_labels=True,
                with_colorbar=False, with_borders=True,
                curvature_brightness=1, curvature_contrast=0.25,
                with_figure=True
            )
            ax = fig.axes[0]
            ax.axis([xmin, xmax, ymin, ymax])
            ax.set_position([0, 0, 1, 1])
            ax.axis('off')
            fig.set_size_inches(5, 5)

            out_path = "{}/{}_{}_{}.pdf".format(flatmaps_dir, subject, task_name, zoom)
            plt.savefig(out_path, bbox_inches='tight', pad_inches=0, dpi=500)
            plt.close(fig)


sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-11
sub-12
sub-13
sub-14
sub-17
sub-20
sub-21
sub-22
sub-23
sub-24
sub-25
